<a href="https://colab.research.google.com/github/sijan2003/Python/blob/master/sentiment_analysis_project_sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#installing kaggle library
! pip install kaggle

let's upload our kaggle.json file


In [3]:

#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing twitter sentiment analysis dataset from kaggle


In [4]:
#API to fetch dataset from kaggle
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 179MB/s]
100% 80.9M/80.9M [00:00<00:00, 180MB/s]


In [5]:

#unzipping the zip files
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing dependencies


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re #re stands for regular expressions
from nltk.corpus import stopwords #nltk stands for natural language tool
from nltk.stem.porter import PorterStemmer #porterstemmer reduces word into their base form
from sklearn.feature_extraction.text import TfidfVectorizer #converts textual data to numerical data
from sklearn.model_selection import train_test_split #splits original data into training and testing data
from sklearn.linear_model import LogisticRegression  # machine learning model used
from sklearn.metrics import accuracy_score


In [7]:
import nltk
nltk.download('stopwords')
"""downloads the "stopwords" dataset from the Natural Language Toolkit (NLTK) library.
 Stopwords are common words (such as "the", "is", "in") that are usually filtered out
  in text processing and analysis because they carry less meaningful information.
This dataset is often used in natural language processing (NLP) tasks to remove these common words from the text."""

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'downloads the "stopwords" dataset from the Natural Language Toolkit (NLTK) library.\n Stopwords are common words (such as "the", "is", "in") that are usually filtered out\n  in text processing and analysis because they carry less meaningful information.\nThis dataset is often used in natural language processing (NLP) tasks to remove these common words from the text.'

just for knowledge we are giving an example


In [8]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data processing


In [9]:
#loading the data from csv file to pandas dataframe
twitter_data =pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
twitter_data.head(5)


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [11]:
#naming the columnns and reading the dataset again
column_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)
twitter_data.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
# counting the missing values in this dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [13]:
#checking the distribution of target column
twitter_data['target'].value_counts()


,count
target,
0,800000
4,800000



converting the target '4' to '1'

In [14]:
twitter_data['target'].replace({4:1},inplace=True)


In [15]:
#checking the distribution of the target column again to check if the value is replaced successfully or not
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0------> negative tweet


1------> positive tweet





**STEMMING**

stemming is the process of reducing a word to it's Root word

for example : actor, actress, acting= act

In [19]:
port_stem=PorterStemmer()

In [20]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  """filters out stopwords from stemmed_content,
  applies stemming to the remaining words, and updates stemmed_content with the processed list."""

  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [18]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head(5)


,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [23]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [24]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [25]:
#separating data and label
x=twitter_data['stemmed_content'].values
y=twitter_data['target'].values
print(x)
print(y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


In [26]:
#splittinng the data to training data and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
""" stratify ----- Statistics: Means to divide a sample into subgroups or strata to ensure that
 various segments of a population are represented proportionally in research or analysis.

 so as to ensure the negative and positive tweets are uniformly distributed and there is no dominance of
 one data"""

' stratify ----- Statistics: Means to divide a sample into subgroups or strata to ensure that\n various segments of a population are represented proportionally in research or analysis.\n \n so as to ensure the negative and positive tweets are uniformly distributed and there is no dominance of \n one data'

In [27]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [28]:

#converting the textual data into numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x_train)  # Learn the vocabulary and IDF from the training data
x_train=vectorizer.transform(x_train)
x_test=vectorizer.transform(x_test)


Why Use TF-IDF?

Term Frequency (TF): Measures how frequently a term occurs in a document. The higher the frequency, the higher the score.

Inverse Document Frequency (IDF): Measures how important a term is. If a term appears in many documents, its IDF score is lower, which means it’s less informative.

TF-IDF: Combines both TF and IDF to give a score for each term in each document, which helps in emphasizing terms that are important (high TF in a few documents) and reducing the weight of terms that appear frequently across all documents (common words).

In [29]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 235045)	0.41996827700291095
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (1, 160636)	1.0
  (2, 443430)	0.3348599670252845
  (2, 433560)	0.3296595898028565
  (2, 409143)	0.15169282335109835
  (2, 407301)	0.18709338684973031
  (2, 406399)	0.32105459490875526
  (2, 288470)	0.16786949597862733
  (2, 266729)	0.24123230668976975
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 129411)	0.29074192727957143
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (2, 77929)	0.31284080750346344
  (3, 411528)	0.27089772444087873
  (3, 406399)	0.29029991238662284
  (3, 388626)	0.3940776331458846
  (3, 172421)	0.37464146922154384
  :	:
  (1279996, 373144)	0.35212500999832036
  (1279996, 318303)	0.21254698865277744
  (1279996, 291078)	0.17981734369155505
  (1279996, 238078)	0.5606696159563151
  (1279996, 2

In [30]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

In [31]:
#training the machine learning model
model=LogisticRegression(max_iter=1000)

In [32]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

MOdel evaluation

In [33]:
#accuracy score
#accuracy on training data
x_train_prediction=model.predict(x_train)
training_data_accuracy=accuracy_score(x_train_prediction,y_train)

In [34]:
print('Accuracy on training data :',training_data_accuracy)

Accuracy on training data : 0.8102125


In [35]:
#accuracy on test data
x_test_prediction=model.predict(x_test)
test_data_accuracy=accuracy_score(x_test_prediction,y_test)
print('Accuracy on test data :',test_data_accuracy)

Accuracy on test data : 0.778021875


Model accuracy= 0.77

In [36]:
#saving the trained model
import pickle
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [37]:
#loading the saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [40]:
x_new=x_test[150]
print(y_test[150])
prediction=loaded_model.predict(x_new)
print(prediction)
if prediction[0]==0:
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet
